In [10]:
import numpy as np
import datetime
import mne
import pickle

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from mne.decoding import cross_val_multiscore, LinearModel, \
                         GeneralizingEstimator, Scaler, Vectorizer

from sklearn.model_selection import StratifiedKFold, cross_val_score, StratifiedShuffleSplit, \
                                    RepeatedStratifiedKFold

In [6]:
class arguments:
    def __init__(self, cond_block, cond_decoding, applyBaseline_bool, n_splits):
        self.cond_block = cond_block #'later' #{early,later}
        self.cond_decoding = cond_decoding #'removeevoked' #{none,removeevoked,resampled}
        self.applyBaseline_bool = applyBaseline_bool #'False'
        self.n_splits = n_splits#100

        self.SAVE_EPOCH_ROOT = '../../data/version5.2/preprocessed/epochs/aft_ICA_rej/'
        self.SAVE_RESULT_ROOT = '../../results/autocorr/eCortex/'
        self.cond_filter ='none' # {none,non_symm}
        self.cond_time = 'prestim' #{prestim,poststim}
        self.subj_num = 1
        self.pre_tmin = -0.4
        self.pre_tmax = 0.05
        self.post_tmin = 0.05
        self.post_tmax = 0.45
        self.num_classes = 2
        self.normalization_type = 'normal'# {normal,lstmPaper}
        self.gen_rand_perm = 0
        self.null_max_iter = 10000
        self.loop_null_iter = 1
        self.gen_decoder_scores = 1
        self.random_state = 42 
        self.max_iter = 10000
        self.n_jobs = 1
        self.scoring = 'roc_auc'
        self.folder='noneFilter_PrePost_decod%s_bsline%s%s/' %(cond_decoding, \
                                                       applyBaseline_bool, \
                                                       n_splits)
#         self.SAVE_RESULT_ROOT = self.SAVE_RESULT_ROOT + self.folder
        print(self.SAVE_RESULT_ROOT)




In [7]:
# SAVE_EPOCH_ROOT = '../../data/preprocessed/epochs/aft_ICA_rej/'
SAVE_EPOCH_ROOT = '../../data/version5.2/preprocessed/epochs/aft_ICA_rej/'
filename_epoch = SAVE_EPOCH_ROOT +  'epochs_sec_applyBaseline_subj1-afterRejICA-epo.fif'

#Read Epochs
epochs_orig = mne.read_epochs(filename_epoch, proj=True, preload=True, verbose=None)
epochs = epochs_orig.copy()
subset = epochs['pred']['non'].copy()
subset = subset.pick_types(eeg=True)
subset.crop(tmin=-0.4,tmax=0.5)

Reading ../../data/version5.2/preprocessed/epochs/aft_ICA_rej/epochs_sec_applyBaseline_subj1-afterRejICA-epo.fif ...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 129) active
    Found the data of interest:
        t =    -400.00 ...    5000.00 ms
        0 CTF compensation matrices available
1197 matching events found
Applying baseline correction (mode: mean)
Adding metadata with 16 columns
Created an SSP operator (subspace dimension = 1)
1 projection items activated


<EpochsFIF  |   876 events (all good), -0.4 - 0.5 sec, baseline [-0.4, 0], ~195.1 MB, data loaded, with metadata,
 'pred/down/non': 216
 'pred/left/non': 224
 'pred/right/non': 219
 'pred/up/non': 217>

In [8]:
if subset['Block==6'].metadata.Ptrn_Type.values.shape[0]>0:
       main_ptrn = subset['Block==6'].metadata.Ptrn_Type.values[0]
else:
       main_ptrn = subset['Block==8'].metadata.Ptrn_Type.values[0]

# only early blocks
subset = subset['Block<7'].copy()
subset = subset['Block>2'].copy()

# Group data based on the previous trial
Grp1 = subset['Trgt_Loc_prev==1'].copy()
Grp2 = subset['Trgt_Loc_prev==2'].copy()
Grp3 = subset['Trgt_Loc_prev==3'].copy()
Grp4 = subset['Trgt_Loc_prev==4'].copy()

Grp1._data = (Grp1._data - np.mean(Grp1._data)) / np.std(Grp1._data)
Grp2._data = (Grp2._data - np.mean(Grp2._data)) / np.std(Grp2._data)
Grp3._data = (Grp3._data - np.mean(Grp3._data)) / np.std(Grp3._data)
Grp4._data = (Grp4._data - np.mean(Grp4._data)) / np.std(Grp4._data)

In [12]:
args = arguments('early', 'none', 'False', 3)
cv = StratifiedShuffleSplit(n_splits=args.n_splits, \
                            random_state=args.random_state)
Grp_data = Grp1

../../results/autocorr/eCortex/


In [13]:
le = LabelEncoder()
clf_SVC  = make_pipeline(
                      StandardScaler(),
                      LinearModel(LinearSVC(random_state=args.random_state,
                                            max_iter=args.null_max_iter)))

X = Grp_data.copy()._data
y = le.fit_transform(Grp_data.copy().metadata.Trgt_Loc_main)

rand_scores = []
rand_diag = []

rand_scores_fit = []
rand_diag_fit = []
print('**************************************************************')
print('Start of the rand loop:\n')
print(str(datetime.datetime.now()))
print('**************************************************************')

for nitr in range(args.loop_null_iter):
    print('**************************************************************')
    print('**************************************************************')
    print('Iteration:\n')
    print(str(datetime.datetime.now()))
    print(nitr)
    print('**************************************************************')
    print('**************************************************************')
    true_Y = y.copy();
    indx = np.random.permutation(true_Y.shape[0]);
    shuffled_Y = true_Y.copy()[indx];
    shuffled_Y = le.fit_transform(shuffled_Y.copy());

    time_gen = GeneralizingEstimator(clf_SVC, scoring=args.scoring,
                                     n_jobs=args.n_jobs, verbose=True)
    # print(np.unique(y))
    # print(np.unique(Grp_data.copy().metadata.Trgt_Loc_main))

    scores = cross_val_multiscore(time_gen, X, shuffled_Y, cv=cv, n_jobs=args.n_jobs)
    scores = np.mean(scores, axis=0) #scores with cv
    scores_diag = np.diag(scores)

    rand_scores.append(scores)
    rand_diag.append(scores_diag)

    # Without using cv, train and test on the same data
    X = Grp_data.copy()._data
    y = le.fit_transform(Grp_data.copy().metadata.Trgt_Loc_main)
    time_gen.fit(X=X ,y=y)
    scores = time_gen.score(X=X, y=y) #scores without cv
    scores_diag = np.diag(scores)

    rand_scores_fit.append(scores)
    rand_diag_fit.append(scores_diag)
print('**************************************************************')
print('End of the rand loop:\n')
print(str(datetime.datetime.now()))
print('**************************************************************')
rand_scores_pck = (rand_scores.copy(), rand_diag.copy())
rand_scores_pck_fit = (rand_scores_fit.copy(), rand_diag_fit.copy())

**************************************************************
Start of the rand loop:

2020-04-13 21:20:14.442753
**************************************************************
**************************************************************
**************************************************************
Iteration:

2020-04-13 21:20:14.443591
0
**************************************************************
**************************************************************
[............................................................] 100.00% Fitting GeneralizingEstimator |
[............................................................] 100.00% Scoring GeneralizingEstimator |
[............................................................] 100.00% Fitting GeneralizingEstimator |
[............................................................] 100.00% Scoring GeneralizingEstimator |
[............................................................] 100.00% Fitting GeneralizingEstimator |
[.............

In [16]:
sc_pck_G1 = rand_scores_pck
sc_pck_fit_G1 = rand_scores_pck_fit

# unpack them
sc_G1, sc_diag_G1 = sc_pck_G1
sc_G2, sc_diag_G2 = sc_pck_G1
sc_G3, sc_diag_G3 = sc_pck_G1
sc_G4, sc_diag_G4 = sc_pck_G1

sc_fit_G1, sc_fit_diag_G1 = sc_pck_fit_G1
sc_fit_G2, sc_fit_diag_G2 = sc_pck_fit_G1
sc_fit_G3, sc_fit_diag_G3 = sc_pck_fit_G1
sc_fit_G4, sc_fit_diag_G4 = sc_pck_fit_G1

In [23]:
sc_G1=np.asarray(sc_G1)
sc_G2=np.asarray(sc_G2)
sc_G3=np.asarray(sc_G3)
sc_G4=np.asarray(sc_G4)

sc_diag_G1=np.asarray(sc_diag_G1)
sc_diag_G2=np.asarray(sc_diag_G2)
sc_diag_G3=np.asarray(sc_diag_G3)
sc_diag_G4=np.asarray(sc_diag_G4)

sc_fit_G1=np.asarray(sc_fit_G1)
sc_fit_G2=np.asarray(sc_fit_G2)
sc_fit_G3=np.asarray(sc_fit_G3)
sc_fit_G4=np.asarray(sc_fit_G4)


sc_fit_diag_G1=np.asarray(sc_fit_diag_G1)
sc_fit_diag_G2=np.asarray(sc_fit_diag_G2)
sc_fit_diag_G3=np.asarray(sc_fit_diag_G3)
sc_fit_diag_G4=np.asarray(sc_fit_diag_G4)


avg_sc= np.zeros([4, args.loop_null_iter, sc_G1.shape[1], sc_G1.shape[2]])
avg_sc[0,:,:,:] = sc_G1
avg_sc[1,:,:,:] = sc_G2
avg_sc[2,:,:,:] = sc_G3
avg_sc[3,:,:,:] = sc_G4
avg_sc = np.mean(avg_sc, axis=0)

avg_diag_sc= np.zeros([4, args.loop_null_iter, sc_diag_G1.shape[1]])
avg_diag_sc[0,:,:]=sc_diag_G1
avg_diag_sc[1,:,:]=sc_diag_G2
avg_diag_sc[2,:,:]=sc_diag_G3
avg_diag_sc[3,:,:]=sc_diag_G4
avg_diag_sc = np.mean(avg_diag_sc, axis=0)

#------ save fit results (no cross validation)
avg_sc_fit= np.zeros([4, args.loop_null_iter, sc_fit_G1.shape[1], sc_fit_G1.shape[2]])
avg_sc_fit[0,:,:,:] = sc_fit_G1
avg_sc_fit[1,:,:,:] = sc_fit_G2
avg_sc_fit[2,:,:,:] = sc_fit_G3
avg_sc_fit[3,:,:,:] = sc_fit_G4
avg_sc_fit = np.mean(avg_sc_fit, axis=0)

avg_diag_sc_fir= np.zeros([4, args.loop_null_iter, sc_fit_diag_G1.shape[1]])
avg_diag_sc_fir[0,:,:]=sc_fit_diag_G1
avg_diag_sc_fir[1,:,:]=sc_fit_diag_G2
avg_diag_sc_fir[2,:,:]=sc_fit_diag_G3
avg_diag_sc_fir[3,:,:]=sc_fit_diag_G4
avg_diag_sc_fir = np.mean(avg_diag_sc_fir, axis=0)

# ------ Pack all scores and save them
sc_subj_pck = [avg_sc, avg_diag_sc, avg_sc_fit, avg_diag_sc_fir]

(5, 226, 226)
(4, 5, 226, 226)
sc_diag_G1 (5, 226)
avg_diag_sc (4, 5, 226)
